In [3]:
!wget -q https://github.com/wncc/Machine-Learning-LS-24/raw/main/Week%202/Assignment/Neural%20Network%20Assignment/homer_bart.zip -O images.zip

In [4]:
!unzip -q images.zip -d images

Importing Libraries

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split

Selecting Device

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Defining our Dataset

In [72]:
class SimpsonsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for label, character in enumerate(["Bart", "Homer"]):
            character_dir = os.path.join(root_dir, character)
            for img_file in os.listdir(character_dir):
                if img_file.endswith((".jpg", ".jpeg", ".png", ".bmp")):
                    self.images.append(os.path.join(character_dir, img_file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = self.images[index]
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(self.labels[index])

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[index], dtype=torch.float32)
        return image, label

Resizing and converting our data to tensor

In [73]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [74]:
dataset = SimpsonsDataset(root_dir="/content/images", transform=transform)

9:1 train:test ratio as asked

In [75]:
test_size = int(0.1 * len(dataset))
train_size = len(dataset) - test_size
train_set, test_set = random_split(dataset, [train_size, test_size])

In [76]:
batch_size = 32
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

Creating our Neural Net Architecture

In [77]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*64*3, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)


    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [78]:
model = SimpleNN().to(device)

Loss and optimizer

In [79]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

Training the Network

In [80]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    losses = []
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device).float().unsqueeze(1)

        # forward
        outputs = model(data)
        loss = criterion(outputs, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

    print(f"Cost at epoch {epoch+1}: {sum(losses)/len(losses):.4f}")

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Cost at epoch 1: 0.6790
Cost at epoch 2: 0.6539
Cost at epoch 3: 0.6388
Cost at epoch 4: 0.6181
Cost at epoch 5: 0.5978
Cost at epoch 6: 0.5723
Cost at epoch 7: 0.5607
Cost at epoch 8: 0.5499
Cost at epoch 9: 0.5323
Cost at epoch 10: 0.5556
Cost at epoch 11: 0.5447
Cost at epoch 12: 0.5319
Cost at epoch 13: 0.4958
Cost at epoch 14: 0.4806
Cost at epoch 15: 0.4913
Cost at epoch 16: 0.4899
Cost at epoch 17: 0.4564
Cost at epoch 18: 0.4552
Cost at epoch 19: 0.4369
Cost at epoch 20: 0.4282
Cost at epoch 21: 0.4244
Cost at epoch 22: 0.4080
Cost at epoch 23: 0.4131
Cost at epoch 24: 0.4311
Cost at epoch 25: 0.4070
Cost at epoch 26: 0.4447
Cost at epoch 27: 0.4568
Cost at epoch 28: 0.4452
Cost at epoch 29: 0.3900
Cost at epoch 30: 0.3797
Cost at epoch 31: 0.3839
Cost at epoch 32: 0.3910
Cost at epoch 33: 0.3621
Cost at epoch 34: 0.3408
Cost at epoch 35: 0.3413
Cost at epoch 36: 0.3303
Cost at epoch 37: 0.3253
Cost at epoch 38: 0.3194
Cost at epoch 39: 0.3219
Cost at epoch 40: 0.3093
Cost at e

Checking accuracy on training to see how good our model is

In [81]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for data, targets in loader:
            data = data.to(device)
            targets = targets.to(device).unsqueeze(1)
            outputs = model(data)
            predictions = (outputs >= 0.5).float()
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

    accuracy = float(num_correct) / float(num_samples) * 100
    print(
          f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
      )
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

Now similarly check accuracy on test data

In [82]:
print("Checking accuracy on Test Set")
accuracy = check_accuracy(test_loader, model)

if accuracy > 80:
    print(f"Passed: Test Accuracy is greater than 80% with a great accuracy of {accuracy:.2f}% !!!")
else:
    print(f"Failed: Test Accuracy is less than 80% and is {accuracy:.2f}%, let's try again to do better :)")

Checking accuracy on Test Set
Got 24 / 26 with accuracy 92.31
Accuracy: 92.31%
Passed: Test Accuracy is greater than 80% with a great accuracy of 92.31% !!!
